In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Exemple pratique d'utilisation des réseaux de neurones graphiques (GNN) avec 1/4 du dataset Cora, nous allons utiliser la bibliothèque PyTorch Geometric.


<br>
<B>Dataset<br>
./cora<br>
___cora.cites<br>
    
___cora.content <br>


L'ensemble de données Cora se compose de 2708 publications scientifiques classées dans l'une des sept classes. Chaque publication de l'ensemble de données est décrite par un vecteur de mots à valeur 0/1 indiquant l'absence ou la présence du mot correspondant dans le dictionnaire. Le dictionnaire comprend 1433 mots uniques.<br>
    <br>
Nœuds = publications (papiers, livres…) <br>
Arrêt = Citations
Caractéristiques du nœud = vecteurs de mots <br>
7 Étiquettes = type de publication par ex. Neural_networks, dule_learning, reinforcement_learning, probabiliste_methods <br>

 Nombre de graphiques: 1

Nombre de fonctionnalités: 1433

Nombre de classes: 7
    

#Étapes pour réaliser le cas pratique

Installation des bibliothèques nécessaires <br>
Chargement et préparation des données <br>
Définition du modèle GNN <br>
Entraînement et évaluation du modèle <br>

#1. Installation des bibliothèques nécessaires

Installation de PyTorch et PyTorch Geometric.

In [1]:
pip install torch torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

#2. Chargement et préparation des données

Nous allons charger le dataset Cora fourni par PyTorch Geometric et utiliser 1/4 de ce dataset pour simplifier l'exemple.

In [28]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
import networkx as nx
import pandas as pd

# Charger le dataset Cora
dataset = Planetoid(root='/content/drive/MyDrive/CoraDataset-main/cora', name='Cora', transform=T.NormalizeFeatures())

# Utiliser 1/4 du dataset
subset_size = len(dataset[0].y) // 4
indices = torch.randperm(len(dataset[0].y))[:subset_size]
mask = torch.zeros_like(dataset[0].y, dtype=torch.bool)
mask[indices] = True

# Masquer les données non utilisées
data = dataset[0]
data.train_mask = mask
data.val_mask = ~mask
data.test_mask = ~mask


In [8]:
print(dataset[0])

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


#3. Définition du modèle GNN

Nous allons utiliser un simple réseau de neurones graphiques à base de Graph Convolutional Network (GCN).

In [4]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


#4. Entraînement et évaluation du modèle

Enfin, nous allons définir les fonctions d'entraînement et d'évaluation, puis entraîner le modèle.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    logits, accs = model(data), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, '
              f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')


Epoch: 010, Loss: 1.8560, Train Acc: 0.2939, Val Acc: 0.3048, Test Acc: 0.3048
Epoch: 020, Loss: 1.7552, Train Acc: 0.2939, Val Acc: 0.3048, Test Acc: 0.3048
Epoch: 030, Loss: 1.6629, Train Acc: 0.3013, Val Acc: 0.3087, Test Acc: 0.3087
Epoch: 040, Loss: 1.5516, Train Acc: 0.3973, Val Acc: 0.3742, Test Acc: 0.3742
Epoch: 050, Loss: 1.4120, Train Acc: 0.5569, Val Acc: 0.5382, Test Acc: 0.5382
Epoch: 060, Loss: 1.2570, Train Acc: 0.7016, Val Acc: 0.6805, Test Acc: 0.6805
Epoch: 070, Loss: 1.1087, Train Acc: 0.7386, Val Acc: 0.7223, Test Acc: 0.7223
Epoch: 080, Loss: 0.9801, Train Acc: 0.7903, Val Acc: 0.7553, Test Acc: 0.7553
Epoch: 090, Loss: 0.8710, Train Acc: 0.8464, Val Acc: 0.7991, Test Acc: 0.7991
Epoch: 100, Loss: 0.7795, Train Acc: 0.8774, Val Acc: 0.8232, Test Acc: 0.8232
Epoch: 110, Loss: 0.7042, Train Acc: 0.9055, Val Acc: 0.8355, Test Acc: 0.8355
Epoch: 120, Loss: 0.6433, Train Acc: 0.9129, Val Acc: 0.8439, Test Acc: 0.8439
Epoch: 130, Loss: 0.5942, Train Acc: 0.9188, Val Acc

#Prédictions :

Nous faisons des prédictions sur les données de test. <br>
Nous comparons les étiquettes prédites aux vraies étiquettes et affichons les résultats.<br>
Nous visualisons le graphe en coloriant les nœuds correctement et incorrectement prédits.<br>

In [20]:
# Faire des prédictions
model.eval()
with torch.no_grad():
    logits = model(data)
    test_mask = data.test_mask
    pred = logits[test_mask].max(1)[1]
    test_labels = data.y[test_mask]
    correct = pred.eq(test_labels).sum().item()
    total = len(test_labels)
    accuracy = correct / total
    Predicted_labels = pred.cpu().numpy()
    True_labels = test_labels.cpu().numpy()
    print("Accuracy:", accuracy)
    print("Predicted labels:", pred.cpu().numpy())
    print("True labels:", test_labels.cpu().numpy())
    print("Correct:", correct)
    print("Total:", total)
    print("Incorrect:", total - correct)
    print("Incorrect percentage:", (total - correct) / total * 100)
    print("Correct percentage:", correct / total * 100)
    print("\n")

# Visualisation des résultats
# Create a color map for all nodes, not just the test set
all_pred = logits.max(1)[1]
all_labels = data.y
correct = all_pred.eq(all_labels).cpu().numpy()
color_map = ['red' if not c else 'green' for c in correct]  # Adjust color_map to cover all nodes

plt.figure(figsize=(100, 60))
nx.draw(nx.from_edgelist(data.edge_index.cpu().numpy().T),
        node_color=color_map,
        with_labels=True,
        node_size=1000,
        cmap=plt.cm.RdYlBu)
plt.title("Graph nodes (Green: Correct, Red: Incorrect)")
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [27]:
print(Predicted_labels.shape)
print(True_labels.shape)
print("True labels:", True_labels)

(2031,)
(2031,)
True labels: [4 4 0 ... 3 3 3]


In [34]:
df_Predicted_labels_vs_True_labels = pd.DataFrame({'Predicted_labels': Predicted_labels, 'True_labels': True_labels})
#print(df_Predicted_labels_vs_True_labels)
df_Predicted_labels_vs_True_labels.to_csv('Predicted_labels_vs_True_labels.csv', index=False)
df_Predicted_labels_vs_True_labels.head(20)



,Predicted_labels,True_labels
0,4,4
1,3,4
2,0,0
3,3,3
4,2,2
5,0,0
6,0,3
7,3,3
8,0,0
9,0,0
